ELEC-E5550 - Statistical Natural Language Processing
# SET 2: $n$-gram language models

# Released: 23.1.2024
# Deadline: 2.2.2024

In [1]:
%%capture
!pip install nose==1.3.7

# Overview
After completing this assignment, you'll understand how statical language models can be estimated. You'll be able to evaluate them and to generate text using them.


# Table of contents

* [Introduction](#intro)
    * [Language models](#languagemodel)
    * [$n$-gram language models](#ngramlm)
* [Task 1: Counting $n$-grams](#task_1)
    * [Step 1.1: Pad with sentence boundary symbols](#subtask_1_1)
    * [Step 1.2: Forming $n$-grams](#subtask_1_2)
    * [Step 1.3: $n$-gram pipeline](#subtask_1_3)
    * [Step 1.4: Count $n$-grams](#subtask_1_4)
    * [Step 1.5: Full counting pipeline](#subtask_1_5)
* [Task 2: Estimating an $n$-gram language model](#task_2)
    * [Step 2.1: MLE](#subtask_2_1)
    * [Step 2.2: Smoothing](#subtask_2_2)
* [Task 3: Evaluating a language model](#task_3)
    * [Step 3.1: Perplexity](#subtask_3_1)
* [Task 4: Sampling from a language model](#task_4)
    * [Step 4.1: Generate text](#subtask_4_1)
* [Task 5: Working with real data](#task_5)
    * [Step 5.1: Limiting the vocabulary](#subtask_5_1)
    * [Step 5.2: Counting pipeline on real data](#subtask_5_2)
    * [Step 5.3: Generate sentences, comment on differences](#subtask_5_3)
* [Checklist before submission](#checklist)


## Introduction <a class="anchor" id="intro"></a>
## Language models <a class="anchor" id="languagemodel"></a>
As you've already noticed in the first assignment, different language sequences are not equally likely to occur. We've only looked at word frequencies and at frequencies of letter sequences, but what if we want to estimate how probable it is to see some sentence? Well, for this purpose you'll need a **language model**.

A **language model** predicts the following word (or other symbol) given the observed history.
$P(w_i \mid w_1, ..., w_{i−1})$.
**Language models** are useful, for example, in the task of speech recognition. They help to distinguish between homophones (words that sound the same but have different meanings), for example, _"too"_ and _"two"_ in _"I love you too"_ and _"I love you two"._

### What is an $n$-gram language model? <a class="anchor" id="ngramlm"></a>
An **$n$-gram language model** approximates the probability of a word given all the previous words by using only the conditional probability of the $n-1$ preceding words. This approach is based on the Markov assumption: the next word depends only on a fixed-size window of previous words and not on the whole history: $P(w_i| w_{i-n+1},...,w_{i−1})$. 
To use an $n$-gram language model, we need an estimate of the probability of seeing a particular word given the recent history. For instance, in the 3-gram model, the history is 2 words.

* 3-gram: (say hello **to**) 
* 2 words of history
* 1 word for **prediction**


The simplest way to estimate probabilities is **maximum likelihood estimation** (MLE). To get the MLE estimate of an $n$-gram model we get counts from a corpus and normalize these counts to lie between 0 and 1. In the case of bigram language model, when we want to get a probability of some particular bigram $P(w_i \mid w_{i−1})$, we’ll compute the count of the bigram $C(w_{i−1}w_i)$ and normalize it by the sum of the counts of all the bigrams that start with the same first word $w_{i−1}$. It is easy to notice that the sum of all bigram counts that have the same fist word is simply the unigram count for that word $w_{i−1}$. Thus, we get:

$ P(w_i \mid w_{i−1}) = \dfrac{C(w_{i−1}w_i)}{C(w_{i−1})}, $ where
$C$ is the number of occurrences in the training set.

The probability of the entire word sequence can be computed using **the chain rule of probability**. And considering the bigram assumption, it is:

$$P(w_1^n) ≈ \prod_{i=1}^{n}P(w_i|w_1^{i−1}) ≈ \prod_{i=1}^{n}P(w_i|w_{i−1}),$$
where we use the subscript and superscript to denote the lower and upper bounds of the word indices; the notation $w_1^n$ means the words from $w_1$ to $w_n$.



## TASK 1 <a class="anchor" id="task_1"></a>
## Counting $n$-grams 
### 1.1 Pad with sentence boundary symbols (1 Point) <a class="anchor" id="subtask_1_1"></a>

When dealing with language, it is very important to know what words tend to start a sentence and what words tend to end it. To learn this with $n$-grams, we create special symbols for start-of-sentence  _"&lt;s>"_ and end-of-sentence _"&lt;/s>"_. As a pre-processing step, you need to pad your sentences with these symbols.

The number of sentence start tokens depends on $n$, the $n$-gram order. At the start of the sentence, the only context should be sentence start - so we use $n-1$ sentence start tokens. One sentence-end token is enough for any $n$, because we only need to predict one.

Write a function that takes a list of tokens and pads it with sentence boundary symbols.

In [1]:
def pad(tokens, n):
    """Takes an iterable of tokens and pads with sentence boundary symbols.
    
    Always adds sentence end symbols. 
    For unigram sequences, does not add sentence starts.
    
    Arguments
    ---------
    tokens : list, tuple, iterable
        sentence to be padded
    n : int
        the ngram order
    
    Returns
    -------
    tuple
        Input padded with sentence boundary symbols
    """
    start = "<s>"
    end = "</s>"
    # Always return tuples, we don't want to modify the input in-place.
    tokens = tuple(tokens)  
    # YOUR CODE HERE
    if n > 1:
        tokens = (start,) * (n - 1) + tokens
    tokens = tokens + (end,)
    return tokens
    # raise NotImplementedError()

In [7]:
from nose.tools import assert_equal

assert_equal(pad(['a','b','c'], 2), ('<s>', 'a', 'b', 'c', '</s>'))
assert_equal(pad(('a','b','c'), 2), ('<s>', 'a', 'b', 'c', '</s>'))
assert_equal(pad(iter('abc'), 3), ('<s>', '<s>', 'a', 'b', 'c', '</s>'))
assert_equal(pad(('a','b','c'), 1), ('a', 'b', 'c', '</s>'))


### 1.2 Forming $n$-grams (1 Point) <a class="anchor" id="subtask_1_2"></a>

Now we create a function that takes a list of tokens and forms all the $n$-grams that it can. (A,B,C) can form the bigrams (A,B) and (B,C)

In [11]:
def make_n_grams(tokens, n):
    """Takes in a tuple of tokens and forms n-grams
    
    Arguments
    ---------
    tokens : tuple
        Tokens to make ngrams from
    n : int
        The order of ngrams to make
    Returns
    -------
    list
        A list of tuples: all ngrams of the specified order.
    """
    # YOUR CODE HERE
    n_grams = []
    for i in range(len(tokens) - n + 1):
        n_gram = tokens[i:i + n]
        n_grams.append(n_gram)
    return n_grams
    # raise NotImplementedError()

In [12]:
from nose.tools import assert_equal

assert_equal(make_n_grams(('a','b','c','d','e'), 2), [('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e')])
assert_equal(make_n_grams(('a','b','c','d','e'), 3), [('a', 'b', 'c'), ('b', 'c', 'd'), ('c', 'd', 'e')])
assert_equal(make_n_grams(('a','b','c','d','e'), 1), [('a',), ('b',), ('c',), ('d',), ('e',)])

### 1.3 $n$-gram pipeline (1 Point) <a class="anchor" id="subtask_1_3"></a>

Now we can integrate all the functions so far into an $n$-gram iterating pipeline. The pipeline produces all possible $n$-grams up to order $n$.

In this task, the function is already implemented for you, so you don't need to implement anything here. However, the function combines your previous implementations, so it acts as an additional test. 

We will use this pattern multiple times in this assignment: integration is already implemented for you, so that the tasks that you need to implement don't depend on anything else.

In [13]:
def allgrams_pipeline(data, max_n):
    """Produces ngrams of all orders up to max_n from data, with padding
    
    This uses the user defined pad() and make_n_grams() functions.
    It acts as an additional test for those.
    
    However, you must not change this. If there is some error, change 
    pad or make_n_grams instead."""
    for sentence in data:
        for n in range(1, max_n+1):
            padded = pad(sentence, n)
            yield from make_n_grams(padded, n)
    return

In [14]:
# Small sanity check for allgrams_pipeline:
iterator = allgrams_pipeline([[1,2,3,],[3]], 2)
answers = [(1,), (2,), (3,), 
           ("</s>",), ("<s>", 1), (1, 2), (2, 3), (3, "</s>"), 
           (3,), ("</s>",), 
           ("<s>", 3), (3, "</s>")]
for pipeline_produced, answer in zip(iterator, answers):
    assert_equal(pipeline_produced, answer)

### 1.4 Count $n$-grams (3 Points) <a class="anchor" id="subtask_1_4"></a>
Now we can collect statistics from a whole corpus of text.

In [18]:
from collections import Counter, defaultdict

def get_counts(ngrams, max_n):
    """Counts n-grams in a dataset.
    
    Takes in a stream of ngrams, for example:
    
    ('this',),
    ('is',), 
    ('the',), 
    ('first',), 
    ('sentence',),
    ('</s>',),    
    ('<s>', 'this'), 
    ('this', 'is'), 
    ('is', 'the'), 
    ('the', 'first'), 
    ('first', 'sentence'), 
    ('sentence', '</s>'),

    Note how the stream has a mix of unigrams and bigrams.

    The output is a triply nested dict.
    The first level is indexed by ngram order,
    the second level is indexed by the history,
    and the third level is indexed by the last token (the predicted token).
    Additionally, we recommend making the third level an extended type of dict: a Counter
    See https://docs.python.org/3/library/collections.html#collections.Counter
    Example of the output structure:
    {
        1: {
            (): 
                Counter({
                    '<s>': 21,
                    'this': 43,
                    'most': 31,
                    'is': 50,
                })
        2: {
            ('<s>', ): 
                Counter({
                    'this': 21,
                }),
            ('the',):
                Counter({
                    'most': 31,
                    'least': 14,
                }),
        3: {
            ('<s>', 'this'): 
                Counter({
                    'is': 12,
                    'has': 8,
                    '</s>': 1,
                }),
            ('the', 'most'):
                Counter({
                    'beautiful': 8,
                    'intelligent': 10,
                    'funny': 3,
                }),
    }
    This structure is useful, since each history will also get its own 
    conditional probability distribution.
    Note that when n==1, the n-gram history simply becomes 
    the empty tuple, (). This is fine.
  
    Arguments
    ---------
    sentences : iterable (such as list)
        An iterable over ngrams.
    max_n : int
        The maximum ngram order.
        
    Returns
    -------
    dict
        Triply nested dict, from n-gram order to n_gram history parts, 
        to a dictionary of all continuations and their counts, e.g.
        {2: {('a',): {'b': 3 'c': 4}}}
    
    """
    
    n_gram_dict = {order: defaultdict(Counter) for order in range(1,max_n+1)}
    # The line above creates the triply nested dict.
    # The second and third layers are special: defaultdict and Counter
    # See their documentation:
    # https://docs.python.org/3/library/collections.html#collections.defaultdict
    # https://docs.python.org/3/library/collections.html#collections.Counter
    
    for ngram in iter(ngrams):
        # YOUR CODE HERE
        # raise NotImplementedError()
        length = len(ngram)
        if length <= max_n:
            history = ngram[:-1]
            token = ngram[-1]
            n_gram_dict[length][history][token] += 1
    
    # Lastly, make the defaultdicts into normal dicts, 
    # so that defaultdict doesn't bite us later (it can hide some bugs)
    return {n: dict(counts) for n, counts in n_gram_dict.items()}

In [19]:
dummy_corpus = [
                ("say",), 
                ("hello",), 
                ("to",), 
                ("my",),
                ("little",),
                ("friend",),
                ("</s>",),
                ("<s>", "say"), 
                ("say", "hello"), 
                ("hello", "to"), 
                ("to", "my"),
                ("my", "little"),
                ("little", "friend"),
                ("friend", "</s>"),
                ("<s>", "say", "hello"), 
                ("say", "hello", "to"), 
                ("hello", "to", "my"), 
                ("to", "my", "little"),
                ("my", "little", "friend"),
                ("little", "friend", "</s>"),
                ("say",),
                ("hello",),
                ("</s>",),
                ("<s>", "say"),
                ("say", "hello"),
                ("hello", "</s>"),
                ("<s>", "say", "hello"),
                ("say", "hello", "</s>"),
                ("say",),
                ("it",),
                ("to",),
                ("my",),
                ("hand",),
                ("</s>",),
                ("<s>", "say"),
                ("say", "it"),
                ("it", "to"),
                ("to", "my"),
                ("my", "hand"),
                ("hand", "</s>"),
                ('<s>', '<s>', 'say'),
                ("<s>", "say", "it"),
                ("say", "it", "to"),
                ("it", "to", "my"),
                ("to", "my", "hand"),
                ("my", "hand", "</s>"),
]

dummy_model = get_counts(dummy_corpus, 3)
assert_equal(dummy_model[3][('<s>', 'say')], {'hello': 2, 'it': 1})
assert_equal(dummy_model[3][('my', 'hand')], {'</s>': 1})
assert_equal(dummy_model[2][('my',)], {'little': 1, 'hand': 1})
assert_equal(dummy_model[2][('<s>',)], {'say': 3})
assert_equal(dummy_model[2][('<s>',)], {'say': 3})
assert '<s>' not in dummy_model[1][tuple()]
assert dummy_model[1][tuple()]['say'] == 3


### 1.5 Full counting pipeline (1 Point) <a class="anchor" id="subtask_1_5"></a>

Now we can integrate everything so far: the $n$-gram pipeline and the counting function. Put together, these take a tokenized corpus and produce counts of all possible $n$-grams upto order n.
They are designed to work together:
```python
get_counts(allgrams_pipeline(corpus, n), n)
```

In [21]:
# Putting everything so far together:

dummy_data = [["say", "hello", "to", "my", "little", "friend"],
              ["say", "hello"],
              ["say", "it", "to", "my", "hand"]]
dummy_model = get_counts(allgrams_pipeline(dummy_data, 3), 3)
assert_equal(dummy_model[3][('<s>', 'say')], {'hello': 2, 'it': 1})
assert_equal(dummy_model[3][('my', 'hand')], {'</s>': 1})
assert_equal(dummy_model[2][('my',)], {'little': 1, 'hand': 1})

## TASK 2 <a class="anchor" id="task_2"></a>
## Estimating an $n$-gram language model 

Now that we can count $n$-grams, we'll start estimating language models with them. In practice this means taking the counts and producing probability estimates.

### 2.1 MLE (3 Points)  <a class="anchor" id="subtask_2_1"></a>

We'll look at the **Maximum Likelihood Estimate** (MLE) for bigrams. Let's use the subscript and superscript to denote the lower and upper bounds of the word indices; the notation $w^{i−1}_{i−n+1}$ denotes the $n-1$ words before $w_{i}$. In general MLE works like this:

$$P(w_i \mid w^{i−1}_{i−n+1}) = \frac{C(w^{i−1}_{i−n+1}w_i)}{C(w^{i−1}_{i−n+1})},$$ where $C(x)$ is the number of occurrences of $x$ in the training corpus. Basically, you just divide the count of a particular $n$-gram by the count of its context (history) part. 

* 3-gram: (say hello **to**) 
* 2 words of context
* 1 word for **prediction**

Here, the whole 3-gram is (say hello to), and its context part is (say hello). If some $n$-gram is absent from the training corpus, its MLE estimate would be zero.

#### Log-domain
Before we get to computing though, there is one important thing to consider.

Probabilities are small numbers, between 0 and 1, and they often need to be multiplied, which makes them smaller still. This quickly leads to numerical instability. In the case of $n$-gram language models, we encounter this when estimating a probability of a long sentence (we need to multiply many $n$-gram probabilities together). 

Typically probability computations are done in the log-domain instead. In the log-domain, multiplication becomes addition. This solves numerical stability, but also makes many formulas much simpler and faster.

With log-probabilities, we define $log(0) = -\infty$

In [24]:
"""
This cell has a utility function, which you need to use down the line.
The function is already provided here because it is also needed for the
sanity checks in the visible tests for the next task.
"""

def logsumexp2(*logs):
    """Linear-scale addition in log-scale
    
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations"""
    x_star = max(logs)
    return x_star + log2(sum(pow(2, x-x_star) for x in logs))

In [35]:
from math import log2
NEGINF = -float('inf')

def logprob_mle(counts, context, token):
    """Produces Maximum Likelihood Estimate of log(P(token | context))
    
    The ngram counts are as produced by get_counts
    Example of the model_counts structure:
    {
        1: {
            (,): 
                {
                    '<s>': 21,
                    'this': 43,
                    'most': 31,
                    'is': 50,
                })
        2: {
            ('<s>', ): 
                {
                    'this': 21,
                }),
            ('the',):
                {
                    'most': 31,
                    'least': 14,
                }),
        3: {
            ('<s>', 'this'): 
                {
                    'is': 12,
                    'has': 8,
                    '</s>': 1,
                }),
            ('the', 'most'):
                {
                    'beautiful': 8,
                    'intelligent': 10,
                    'funny': 3,
                }),
    }
    
    What should be done if the context has not been seen?
    In this case, we have no definition for the distribution P(x | context).
    Here we take the choice that we find the highest order of ngram for which
    the context has been seen.

    What should be done if the token has not been seen?
    This can lead to a ValueError("math domain error"), due to taking a
    logarithm of 0. In this case, return NEGINF (defined above).
    
    TIP: If you find yourself with a division by zero error, you should instead
    use logarithm identities to convert the division to substraction.
    TIP2: You might be masking the above error if you use a try-except for NEGINF.
    TIP3: To get the context counts, use sum(counts[n][context].values())

    Arguments
    ---------
    counts : dict
        Triply nested dict as shown above.
    context : tuple
        The context to predict on as tuple, e.g. ('<s>',)
    token : str
        The token to predict.
        
    Returns
    -------
    float
        The log probabilty maximum likelihood estimate
    """
    
    # Find an order where context has been seen: 
    n = len(context) + 1
    if n not in counts or context not in counts[n]:
        if n == 1:
            raise ValueError("Invalid counts-dict, needs to have all lower order counts.")
        return logprob_mle(counts, context[1:], token)
    
    # Then compute the log probability.
    # Use the log2() function, i.e. use 2 base logarithms
    # YOUR CODE HERE

    # Get the count of the token given the context
    count = counts[n][context].get(token, 0)
    
    if count > 0:
        total_context_count = sum(counts[n][context].values())
        return log2(count) - log2(total_context_count)
    else:
        return NEGINF
    
    # raise NotImplementedError()

In [36]:
"""
This cell contains a premade n-gram count dict, in the same format
as returned by get_counts(). This is used for a few tests below.
Do not change this; changing this would only make it impossible for
you to benefit from the provided visible tests.
"""

from collections import Counter
TEST_COUNTS = {1: 
                    {
                     tuple():
                        Counter({'say': 3, 
                         'hello': 2, 
                         'to': 2, 
                         'my': 2, 
                         'little': 1, 
                         'friend': 1, 
                         '</s>': 3, 
                         'it': 1, 
                         'hand': 1})
                    }, 
                2: 
                    {
                     ('<s>',): 
                         Counter({'say': 3}), 
                     ('say',): 
                         Counter({'hello': 2, 'it': 1}), 
                     ('hello',): 
                         Counter({'to': 1, '</s>': 1}), 
                     ('to',): 
                         Counter({'my': 2}), 
                     ('my',): 
                         Counter({'little': 1, 'hand': 1}), 
                     ('little',): 
                         Counter({'friend': 1}), 
                     ('friend',): 
                         Counter({'</s>': 1}), 
                     ('it',): 
                         Counter({'to': 1}), 
                     ('hand',): 
                         Counter({'</s>': 1})
                    }, 
                3: 
                    {
                     ('<s>', '<s>'): 
                         Counter({'say': 3}), 
                     ('<s>', 'say'): 
                         Counter({'hello': 2, 'it': 1}), 
                     ('say', 'hello'): 
                         Counter({'to': 1, '</s>': 1}), 
                     ('hello', 'to'): 
                         Counter({'my': 1}), 
                     ('to', 'my'): 
                         Counter({'little': 1, 'hand': 1}), 
                     ('my', 'little'): 
                         Counter({'friend': 1}), 
                     ('little', 'friend'): 
                         Counter({'</s>': 1}), 
                     ('say', 'it'): 
                         Counter({'to': 1}), 
                     ('it', 'to'): 
                         Counter({'my': 1}), 
                     ('my', 'hand'): 
                         Counter({'</s>': 1})
                    }
               }

In [37]:
from numpy.testing import assert_almost_equal
from collections import Counter

assert_almost_equal(logprob_mle(TEST_COUNTS, ("hello",), "</s>"), -1.0, 2)
assert_almost_equal(logprob_mle(TEST_COUNTS, ("<s>", "say"), "it"), -1.5850, 2)
assert_almost_equal(logprob_mle(TEST_COUNTS, ("<s>",), "say"), 0., 2)
assert_almost_equal(logprob_mle(TEST_COUNTS, ("say", "it"), "friend"), -float('inf'), 2)
assert_almost_equal(logprob_mle(TEST_COUNTS, ("hello", "to"), "my"), 0., 2)
# "wow" is never in context so this goes all the way to unigram:
assert_almost_equal(logprob_mle(TEST_COUNTS, ("say", "wow"), "say"), -2.4150, 2)


# Sanity check for proper distributions:
# Probability distributions need to sum to 1 (which is 0. in log-domain)
vocab = set(TEST_COUNTS[1][tuple()])
test_contexts = [("hello",), ("<s>",), ("<s>", "say"), ("hello", "to"), ("say", "it"), ("say", "wow")]
for context in test_contexts:
    all_log_probs = [logprob_mle(TEST_COUNTS, context, token)
                    for token in vocab]
    assert_almost_equal(logsumexp2(*all_log_probs), 0., 5)


## 2.2  Smoothing (5 Points)  <a class="anchor" id="subtask_2_2"></a>
Simple $n$-gram models have one very serious limitation: they are unable to give a probability estimate not only for $n$-grams with new out-of-vocabulary words but also for the $n$-grams with known vocabulary but unseen during training. The higher the $n$, the sparser the data, and the more zero counts there will be. To overcome this problem, we need to redistribute some probability mass from more frequent events and give it to the events we’ve never seen. These techniques are called **smoothing**.

In this assignment, you will implement **Absolute discounting** and **interpolation**. It is a decent smoothing method.

First, we must remove some probability mass, which we can then redistribute. In this method, the probability mass is removed by subtracting an absolute amount,  $\delta$, from each $n$-gram count. Since we remove an absolute number, rather than a proportion, this is called absolute discounting.

Next, we add the removed probability mass back in by interpolating with a less-sparse distribution; in this case simply the lower order $n$-gram models. In the unigram case, we interpolate to the uniform distribution.

$$P(w_i \mid w^{i-1}_{i-n+1}) = \frac{\max\left[C(w_{i-n+1}^i)-\delta,0\right]}{C(w^{i-1}_{i−n+1})} 
    + \lambda(w^{i-1}_{i−n+1})P(w_i \mid w^{i-1}_{i-n+2}), n>1$$
    
$$P(w_i) = \frac{\max\left[C(w_i)-\delta, 0\right]}{C(V)} 
    + \lambda({0})\frac{1}{|V|}, n=1$$
   
$C(V)$ is the total unigram count (number of tokens in the data). $|V|$ is the size of the vocabulary.


$\lambda(x)$ is the interpolation weight, which is the total sum of discount deltas subtracted for this context, normalized by the context count (the denominator for this context). Mathematically:

$$\lambda(w^{i}_{i−n+1}) = 
    \frac{\sum_x\Big( C(w_{i-n+1}^i w_x)-\max\left[C(w_{i-n+1}^iw_x) -\delta, 0\right] \Big)}
    {C(w_{i-n+1}^i)}$$

$$\lambda({0}) = 
    \frac{\sum_x\Big( C(w_x)-\max\left[C(w_x) -\delta, 0\right]\Big)}
    {C(V)}$$
    
Here, in the numerator, $C(w_{i-n+1}^i w_x)-\max\left[C(w_{i-n+1}^iw_x) -\delta, 0\right]$ is the delta that truly was subtracted from $C(w_{i-n+1}^i w_x)$. In an edge case where $C(w_{i-n+1}^i w_x) = 1$ and $\delta=1.4$, this then comes out to 1.


In [49]:
from math import log2
NEGINF = -float('inf')


# Look at logprob_abs_discount first, to understand the full picture.
# Then, start by implementing logprob_discounted
# It has its own tests below; see that you can pass them first.
# Next, implement log_interp_weight.
# It also has its own tests.
# Finally, fill in the missing parts in logprob_abs_discount

def logprob_discounted(counts, context, token, delta):
    """The discounted log probability
        
    Remember to discount to 0 at most, max(count-delta, 0).
    If discounted count becomes 0, the discounted log prob becomes -inf.
    And the same concerns as with logprob_mle apply.
    
    This is the left side of the sum in the probability equations
    (the log version of it).
    """
    n = len(context) + 1  # n-gram order
    # YOUR CODE HERE
    token_count = counts[n][context].get(token, 0)
    discounted_count = max(token_count - delta, 0)
    
    if discounted_count == 0:
        return NEGINF

    context_count = sum(counts[n][context].values())
    return log2(discounted_count) - log2(context_count)
    # raise NotImplementedError()

def log_interp_weight(counts, context, delta):
    """The interpolation weight, as determined by the discount.
    
    You will need to figure out the total sum of discount applied
    for this context.
    
    This is the lambda in the equations (log version of it).
    """
    n = len(context) + 1  # n-gram order
    # YOUR CODE HERE
    total_discount = 0
    
    for token in counts[n][context]:
        count = counts[n][context][token]
        total_discount += delta if count > delta else count
    
    context_count = sum(counts[n][context].values())
    
    if context_count == 0:
        return NEGINF

    return log2(total_discount) - log2(context_count)
    # raise NotImplementedError()
    

def logprob_abs_discount(counts, context, token, delta=0.2):
    """Produces smoothed estimate of log(P(token | context))
    
    Now we will use absolute discounting and interpolation to lower
    orders.
    
    There are four main challenges to compute here:
    1. The discounted count for the token
        - Remember to discount to 0 at most, 
          max(count-delta, 0)
        - If discounted count becomes 0, the discounted log prob becomes -inf.
          And the same concerns as with logprob_mle apply.
    2. The interpolation weight, as determined by the discount.
        - You will need to figure out the total sum of discount applied
          for this context.
    3. The log probability to interpolate with.
        - This is easy: use recursion. So just call:
          logprob_abs_discount(counts, context[1:], token, delta)
        - Unigrams are the special case: they interpolate with the uniform
          distribution P(x) = 1 / vocab size.
    4. Interpolation in the log domain.
        - So you can get log(P_delta(token|context)) and 
          log(P_interp(token|context[1:])) without problems. But then you need
          the logarithmic equivalent of a sum.
        - For that, use the logsumexp2 function defined above. 
    
    The ngram counts are as produced by get_counts, same format as 
    with logprob_mle.
    
    Arguments
    ---------
    counts : dict
        Triply nested dict as shown above.
    context : tuple
        The context to predict on as tuple, e.g. ('<s>',)
    token : str
        The token to predict.
    delta : float
        The value to discount by.
    """
    n = len(context) + 1  # n-gram order
    vocab = set(counts[1][tuple()])
    V = len(vocab)  # Vocabulary size
    
    # Check that word is in the intended vocabulary,
    # i.e. at least seen once in the data (as unigram).
    # If the word is never seen in the data, we cannot expect it.
    if token not in vocab:
        return NEGINF

    # Find an order where context has been seen:     
    if n not in counts or context not in counts[n]:
        if n == 1:
            raise ValueError("Invalid counts-dict, needs to have all lower order counts.")
        return logprob_abs_discount(counts, context[1:], token)
    
    # 1. Discounted prob (computed by separate function above)
    lp_discounted = logprob_discounted(counts, context, token, delta)
    
    # 2. Log interpolation weight (computed by separate function above):
    log_lambda = log_interp_weight(counts, context, delta)
    
    # 3. Log lower order probability:
    if n == 1:
        # Stopping recursion at the unigram level, by interpolating with
        # uniform distribution:
        lp_lower = - log2(V)
    else:  # Recursion
        lp_lower = logprob_abs_discount(counts, context[1:], token)
    
    # 4. Putting it all together:
    # YOUR CODE HERE
    
    # Using logsumexp2 for interpolation
    return logsumexp2(lp_discounted, log_lambda + lp_lower)

    # raise NotImplementedError()

In [44]:
from numpy.testing import assert_almost_equal

# First we test logprob_discounted
# Note that logprob_discounted is not a proper distribution!

assert_almost_equal(logprob_discounted(TEST_COUNTS, ("hello",), "</s>", 0.2), -1.3219, 2)
assert_almost_equal(logprob_discounted(TEST_COUNTS, ("<s>", "say"), "it", 0.2), -1.9069, 2)
assert_almost_equal(logprob_discounted(TEST_COUNTS, ("<s>",), "say", 0.2), -0.0995, 2)
assert_almost_equal(logprob_discounted(TEST_COUNTS, ("say", "it"), "friend", 0.2), -float('inf'), 2)
assert_almost_equal(logprob_discounted(TEST_COUNTS, ("hello", "to"), "my", 0.2), -0.3219, 2)


In [45]:
from numpy.testing import assert_almost_equal
# Then we test log_interp_weight

assert_almost_equal(log_interp_weight(TEST_COUNTS, ("hello",), 0.2), -2.3219, 2)
assert_almost_equal(log_interp_weight(TEST_COUNTS, ("<s>", "say"), 0.2), -2.9069, 2)
assert_almost_equal(log_interp_weight(TEST_COUNTS, ("<s>",), 0.2), -3.9069, 2)
assert_almost_equal(log_interp_weight(TEST_COUNTS, ("say", "it"), 0.2), -2.3219, 2)
assert_almost_equal(log_interp_weight(TEST_COUNTS, ("hello", "to"), 0.2), -2.3219, 2)


In [46]:
from numpy.testing import assert_almost_equal

# Sanity check for proper distributions:
vocab = set(TEST_COUNTS[1][tuple()])
test_contexts = [("hello",), ("<s>",), ("<s>", "say"), ("hello", "to"), ("say", "it"), ("say", "wow")]
for context in test_contexts:
    all_log_probs = [logprob_abs_discount(TEST_COUNTS, context, token)
                    for token in vocab]
    assert_almost_equal(logsumexp2(*all_log_probs), 0., 5)


assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("hello",), "</s>"), -1.1926, 2)
assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("<s>", "say"), "it"), -1.7210, 2)
assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("<s>",), "say"), -0.0803, 2)
assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("say", "it"), "friend"), -8.6439, 2)
assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("hello", "to"), "my"), -0.0255, 2)
# "wow" is never in context so this goes all the way to unigram:
assert_almost_equal(logprob_abs_discount(TEST_COUNTS, ("say", "wow"), "say"), -2.4150, 2)




## TASK 3  <a class="anchor" id="task_3"></a>
## How to evaluate a language model?

The best way to evaluate a language model is to look at its performance in the intended application. Unfortunately, it is usually time-consuming to run the whole system just to test the language model parameters. Instead, we can measure how well an $n$-gram model predicts unseen data called the test set or test corpus. The higher the probability that the model assigns to the test set, the better this model performs.

### 3.1 Perplexity (1 Point) <a class="anchor" id="subtask_3_1"></a>
Perplexity measures the performance of a language model on a test set. Real-world perplexity values typically range from tens to hundreds, and lower is better. There are many ways to view perplexity, see for example [Jurafsky and Martin](https://web.stanford.edu/~jurafsky/slp3/3.pdf). One intuitive view is: if perplexity on a test set is X, the model matches the test data as well as choosing randomly from X tokens would.

If our model assigns zero-probability to some P(Token | Context), but then Context->Token does appear in the test set, perplexity goes to infinity. This is because our model is _certain_ that Context->Token cannot occur. 


Again, the integration is done for you. The implementation of perplexity below uses the log probability functions you have filled in above, and also pad() and make_n_grams(). There is nothing to change here, but the tests serve as additional tests for your implementations. If you get an error here, you probably need to change something in the previous tasks.

In [50]:
def perplexity(test_data, model_counts, logprob_func, **lp_kwargs):
    """
    Computes perplexity on the given test data with the given language model
    (as specified by the counts and the logprob function).
    
    Arguments
    ---------
    test_data : list
        List of lists of tokenized sentences.
    model_counts : dict
        Triply nested dict of n-gram counts, as returned by get_counts()
    logprob_func : function
        Function with signature (counts, context, token), which returns the
        log-probability of the token given the context.
    **lp_kwargs : kwargs
        Log prob key word arguments, passed to logprob_func
    
    Returns
    -------
    float
        The perplexity of the model on the test data.
    """
    max_n = max(model_counts.keys())
    total_log_prob = 0.
    num_tokens = 0
    for sentence in test_data:
        padded = pad(sentence, max_n)
        ngrams = make_n_grams(padded, max_n)
        for *context, token in ngrams:
            total_log_prob += logprob_func(model_counts, tuple(context), token, **lp_kwargs)
            num_tokens += 1
    ppl = pow(2, -total_log_prob / num_tokens)
    return ppl

In [51]:
from numpy.testing import assert_almost_equal

test_data_1 = [("say", "hello", "to", "my", "little", "friend"), ("say", "hello")]
test_data_2 = [("my", "little", "hand"), ("say", "it", "to", "my", "friend")]

assert_almost_equal(perplexity(test_data_1, TEST_COUNTS, logprob_mle), 1.3351, 2)
assert_equal(perplexity(test_data_2, TEST_COUNTS, logprob_mle), float('inf'))

assert_almost_equal(perplexity(test_data_1, TEST_COUNTS, logprob_abs_discount), 1.3683, 2)
assert_almost_equal(perplexity(test_data_2, TEST_COUNTS, logprob_abs_discount), 8.9966, 2)


## TASK 4 <a class="anchor" id="task_4"></a>
## Sampling from a language model
### 4.1 Generate text (Not graded) <a class="anchor" id="subtask_4_1"></a>
Another useful way to make a sanity check of how a model is performing, is by generating sentences with it.
The procedure is about as follows:

1. Start with the appropriate number (n-1) of start symbols "&lt;s>", e.g. ("&lt;s>","&lt;s>") for trigrams. Optionally, some seed text can be given instead.
2. Generate tokens one at a time. Given the context, you get a probability distribution over the next word. Sample from that distribution.
3. The generated tokens become context for the next token to be generated.
4. Stop generating when the end symbol is produced.

NOTE: Although it should be possible to make the pseudorandom behaviour reproducible by setting a seed,
    we were not satisfied with the robustness of that solution. Therefore, the generation functions are not
    autograded. However, they are used later when working with real data.

In [52]:
import random

def generate_text(model_counts, logprob_func, seed_text=None, **lp_kwargs):
    """Generates text from an n-gram model.
    
    Arguments
    ---------
    model_counts : dict
        $n$-gram counts as returned by get_counts()
    logprob_func : callable
        Function with signature (counts, context, token), which returns the
        log-probability of the token given the context.
    seed_text : list, optional
        Text to start generating from. If None, will start from the
        appropriate amount of sentence-start symbols (n-1).
    **lp_kwargs : kwargs
        Log prob key word arguments, passed to logprob_func
        
    Returns
    -------
    tuple
        Sentence generated by model as a list of tokens. If
        seed_text was given, will include it. Padding is stripped.
    """
    max_n = max(model_counts.keys())
    vocab = list(model_counts[1][tuple()])
    if seed_text is None:
        seed_text = ('<s>',) * (max_n-1)
    end = '</s>'
    output = list(seed_text)
    while output[-1] != end and len(output) < 200:  # Also guard against infinite loops
        context = output[-max_n+1:] if max_n > 1 else []
        token_logprobs = [2**logprob_func(model_counts, tuple(context), token, **lp_kwargs) 
                          for token in vocab]
        next_part = random.choices(vocab, token_logprobs)
        output.extend(next_part)  # next_part is a list: [token]
    return tuple(token for token in output if token not in ['<s>', '</s>'])

In [53]:
random.seed(23456)
print("10 sentences from the MLE model:")
for _ in range(10):
    print("\t"+" ".join(generate_text(TEST_COUNTS, logprob_mle)))
print()
print("10 sentences from the Absolute discounted model:")
for _ in range(10):
    print("\t"+" ".join(generate_text(TEST_COUNTS, logprob_abs_discount)))

10 sentences from the MLE model:
	say hello
	say hello
	say hello to my little friend
	say hello to my little friend
	say it to my little friend
	say it to my hand
	say it to my little friend
	say it to my little friend
	say hello to my hand
	say it to my hand

10 sentences from the Absolute discounted model:
	say it say hello
	say hello say hello to my little friend
	say hello to my little friend
	say say it to my little say hello little hello
	say it to my say hello to my little friend
	say hello to my hand
	say hello to my little friend
	say hello
	say it to my hand
	say hello to my hand


## TASK 5 <a class="anchor" id="task_5"></a>
## Working with real data

Let's try to model some real data. In this assignment we are working with Jane Austen's novel Pride and Prejudice, which is provided by Project Gutenberg [here](http://www.gutenberg.org/ebooks/1342). Note that one single novel is actually a very small corpus, but nonetheless it works as an example.

In the 01-text-processing assignment, we saw that text cannot be simply used raw. Instead the text is tokenized and processed to make it suitable for whatever application we have. In the coursedata directory, the novel we are working with has been lowercased, and tokenized, so that some (but not all) punctuation is retained. Each sentence in the novel is put on a separate line. E-book disclaimers and chapter markers have been removed.

In [2]:
janeausten_tokenized = []
with open("../coursedata/ngrams/pride-and-prejudice-normalized-punct.txt") as fi:
    for line in fi:
        janeausten_tokenized.append(line.strip().split())

In [3]:
def detokenize(seq, full_sentence=True):
    """A simple rule-based detokenizer for this assignment"""
    last_seen_start = full_sentence
    formatted_tokens = []
    for token in seq:
        if token == "<s>":
            last_seen_start = True
            continue
        if token == "</s>":
            continue
        if last_seen_start:
            token = token.capitalize()
            last_seen_start = False
        if token == "i":
            token = "I"
        if token in ".!?,;" or token == "'s":
            formatted_tokens.append(token)
        elif formatted_tokens:
            formatted_tokens.append(" " + token)
        else:
            formatted_tokens.append(token)
    return "".join(formatted_tokens)
        

In [4]:
print("Let's see the first sentence of the book:")
print(detokenize(janeausten_tokenized[0]))

Let's see the first sentence of the book:
It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.


### Train-test split

We'll divide the data into training and test sets. The training set is used to create the model, and the test set used for evaluation. Here, we'll select the last 10% of the sentences to be used as the test set.

In [57]:
test_split_index = round(0.9 * len(janeausten_tokenized))
janeausten_train = janeausten_tokenized[:test_split_index]
janeausten_test = janeausten_tokenized[test_split_index:]

print("The training set has", len(janeausten_train), "sentences, and the test set", len(janeausten_test), "sentences")

The training set has 5411 sentences, and the test set 601 sentences


### Vocabulary size

If we just use the full vocabulary of the training data, we will include some rare words, 
while excluding other words. One way to deal with this is to limit the vocabulary to some most common words. 
Everything else will be an unknown token, dealt with, together, as the "&lt;UNK&gt;" token.

We will limit the vocabulary to the 2500 most common words in the training set. Additionally we include the end of sentence tag and the unknown word tag, because we want to predict them.

In [58]:
from collections import Counter
import itertools
janeausten_unigram_counts = Counter(itertools.chain.from_iterable(janeausten_train)).most_common()

print("The 10 most common tokens:")
print("\n".join(word for word, freq in janeausten_unigram_counts[:10]))
print()
print("The 2490-2500 most common tokens:")
print("\n".join(word for word, freq in janeausten_unigram_counts[2490:2500]))

janeausten_vocab_filt = set(word for word, freq in janeausten_unigram_counts[:2500]) | {"</s>", "<unk>"}


The 10 most common tokens:
,
.
the
to
and
of
her
i
a
in

The 2490-2500 most common tokens:
thinks
pompous
promises
writer
punctual
heavy
stately
grievous
summons
viewing


### 5.1 Limiting the vocabulary (1 Point) <a class="anchor" id="subtask_5_1"></a>

Implement the replaing of tokens which are not in the vocabulary (out-of-vocabulary words, OOVs). They are to be replaced with the unknown token "&lt;unk&gt;". Fill in the following function.

In [64]:
def replace_oovs(vocab, data, unk="<unk>"):
    """Replace OOV words with unknown-token
    
    Arguments
    ---------
    vocab : set
        The set of tokens that are in-vocabulary.
        token not in vocab => token is out-of-vocabulary.
    data : list of iterables
        List of sentences, which are lists (or other iterables) of tokens.
    unk : str
        Token to replace tokens which are not in the vocabulary
    
    Returns
    -------
    list
        list of lists, (list of sentences in data, sentences are lists of tokens)
        The data with out-of-vocabulary tokens replaced with the unknown token.
        Does NOT modify in-place.
    
    """
    # NOTE: Do not modify input in-place.
    # YOUR CODE HERE
    # raise NotImplementedError()
    data_oovs_replaced = []

    for sentence in data:
        # Replace each token in the sentence with <unk> if it's not in vocab
        replaced_sentence = [token if token in vocab else unk for token in sentence]
        data_oovs_replaced.append(replaced_sentence)

    return data_oovs_replaced

In [61]:
from nose.tools import assert_equal
assert_equal(replace_oovs({"a","b","c"}, [["a", "b"],["a","b","c","d"]]), [['a', 'b'], ['a', 'b', 'c', '<unk>']])
assert_equal(replace_oovs({"a","b","c"}, [("a", "b"),("a","b","c","d")]), [['a', 'b'], ['a', 'b', 'c', '<unk>']])



Now we'll apply the filter to the Jane Austen data.

In [62]:
janeausten_train_filt = replace_oovs(janeausten_vocab_filt, janeausten_train)
janeausten_test_filt = replace_oovs(janeausten_vocab_filt, janeausten_test)

print("Let's see an example:")
print(detokenize(janeausten_train_filt[2000]))

Let's see an example:
And do you <unk> it to either of those?


### 5.2 Counting pipeline on real data (1 Point) <a class="anchor" id="subtask_5_2"></a>

We'll estimate 5-gram counts from real data. Again, there is nothing for you to implement here, but this is an additional test for your code.


In [63]:

# Produce counts from the real world data:
janeausten_counts = get_counts(allgrams_pipeline(janeausten_train_filt, 5), 5)

# Some more tests based on the real world data:
assert_equal(janeausten_counts[3][("of", "the")]["house"], 13)
assert_equal(janeausten_counts[3][("at", "first")]["."], 4)
assert_equal(janeausten_counts[4][("at", "first", ".")]["</s>"], 4)
assert_equal(janeausten_counts[5][("<s>", "<s>", "<s>", "<s>")]["oh"], 73)


However, even though any unseen token is accounted for by the unknown token, we still get infinite perplexity. Some context-token pairs that occur in the test data have not been seen in the training data, so they get a zero probability estimate.

In [65]:
perplexity(janeausten_test_filt, janeausten_counts, logprob_mle)

inf

### Smoothed model

Now, we'll smooth our counts with a few different delta values and look at perplexities.

Since the smoothed model is a bit more complex, and our Python implementation is not super efficient, we first wrap the log interpolation weight function in a caching class. This class does not change how log_interp_weight functions inside, but it just remembers the solutions that have already been computed, so we save on computing them again.

In [66]:
class LIWCache:
    """Very simple cache for log_interp_weight for speeding up querys
    
    The log_interp_weight function gets called many times with the same
    arguments. The normal Python LRU Cache decorator however cannot handle
    the counts argument, as it is unhashable. 
    """
    def __init__(self, func):
        self.func = func
        self.cache = {}
        self._caching = False
        self.hits = 0
        self.misses = 0
        
    def __call__(self, counts, context, delta):
        if not self._caching:
            return self.func(counts, context, delta)
        key = (context, delta)
        if key not in self.cache:
            self.cache[key] = self.func(counts, context, delta)
            self.misses += 1
        else:
            self.hits += 1
        return self.cache[key]
        
    @property
    def caching(self):
        return self._caching

    @caching.setter
    def caching(self, value):
        self._caching = value
        if not value:
            self.cache = {}  # Empty
            self.hits = 0
            self.misses = 0

if not isinstance(log_interp_weight, LIWCache):
    log_interp_weight = LIWCache(log_interp_weight)

Finally, we can get some type of proper perplexity value. What is more, we can try to optimize the delta value for lower perplexity.

In [67]:
log_interp_weight.caching = True
print("Perplexity with default smoothing:")
print(perplexity(janeausten_test_filt, janeausten_counts, logprob_abs_discount, delta=0.2))
log_interp_weight.caching = False

log_interp_weight.caching = True
print("Perplexity with higher smoothing:")
print(perplexity(janeausten_test_filt, janeausten_counts, logprob_abs_discount, delta=5.2))
log_interp_weight.caching = False


Perplexity with default smoothing:
266.2130040868837
Perplexity with higher smoothing:
225.3559682030848


### 5.3 Generate sentences, comment on differences (3 Points) <a class="anchor" id="subtask_5_3"></a>

Let's see what kinds of sentences do the two types of models generate. We'll seed the generation with the start of the novel, and see where the model goes.
Generate some sentences, and comment on the differences.

- Which model generates text, that is most similar to the original? Why?
- Which model would be the best language model for e.g. optical character recognition of Jane Austen's hand written notes, and why?

NOTE: Answer in the manually graded answer box below.

NOTE2: Generating text with the smoothed model may take some time.

In [68]:
seed_text = ['it', 'is', 'a', 'truth', 'universally', 'acknowledged', ',', 'that',]

In [69]:
print("Sentences from the MLE model:")
print()
for i in range(5):
    print(detokenize(generate_text(janeausten_counts, logprob_mle, seed_text)))


Sentences from the MLE model:

It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.
It is a truth universally acknowledged, that a single man in possession of a most valuable living, had it pleased the gentleman we were speaking of just now.
It is a truth universally acknowledged, that a single man in possession of a most valuable living, had it pleased the gentleman we were speaking of just now.
It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.
It is a truth universally acknowledged, that a single man in possession of a most valuable living, had it pleased the gentleman we were speaking of just now.


In [70]:
print("Sentences from a smoothed model:")
log_interp_weight.caching = True
for i in range(5):
    print(detokenize(generate_text(janeausten_counts, logprob_abs_discount, seed_text, delta=5.2)))
log_interp_weight.caching = False


Sentences from a smoothed model:
It is a truth universally acknowledged, that a mistress might be found for it at this time, when <unk> to by any of those pleasures which too often console the unfortunate for their folly or their <unk>.
It is a truth universally acknowledged, that the living became vacant two years ago, and quitted the house as they entered the woods, and <unk> her most <unk> to his wife he was very fond of them.
It is a truth universally acknowledged, that though she received his attentions with pleasure, she did not suppose lydia to be <unk> <unk>.
It is a truth universally acknowledged, that a person who could <unk> her.
It is a truth universally acknowledged, that the match might be broken in on.


**1. Which model generates text, that is most similar to the original? Why?**

From the examples, we can see that the MLE model generated the most similar text to the original text. This is because the MLE model makes prediction based on the grams that have actually occurred, such as the first sentence in the novel. Since there are probably no other grams that occur exactly like the original sentence, the predicted tokens are simply generated with maximum likelihood. There is no smoothing process, so the most frequently occured grams will dominate the generated sentences, given previous context. MLE never generates sentence whose grams have not been observed before. 

**2. Which model would be the best language model for e.g. optical character recognition of Jane Austen's hand written notes, and why?**

I think that the smooth model is more fitting for optical character recognition (OCR) problem. As OCR scans through the text, it needs to determine the context. The MLE would be heavily biased based on the training corpus and never predicts unknown sequences as they all have zero probability. On the other hand, the smoothing model gives non-zero probability for unknown sequences, making it more flexible to deal with text data that is completely unique from the training corpus

## Checklist before submission <a class="anchor" id="checklist"></a>
### 1
To make sure that you didn't forget to import some package or to name some variable, press **Kernel -> Restart** and then **Cell -> Run All**. This way your code will be run exactly in the same order as during the autograding.
### 2
In this exercise, validation will not work well because the generation will take a long time. You should rely on running all as in **1**
### 3
To submit the notebook, click on the **jupyterhub** logo in the upper left part of the window, choose the **Assignments** folder, and press **submit**. You can submit multiple times, only the last one counts.
### 4
Please fill in the feedback form in the [Assignment](https://mycourses.aalto.fi/mod/questionnaire/view.php?id=1122378) section of Mycoures.